In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('rec').getOrCreate()

In [2]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [3]:
data = spark.read.csv('./datasets/movielens_ratings.csv', inferSchema=True, header=True)

In [4]:
data.show()

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows



In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
train, test = data.randomSplit([0.8, 0.2])

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

In [10]:
model = als.fit(train)

In [11]:
predictions = model.transform(test)

In [12]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    18|  0.8836241|
|     85|   4.0|     7|  2.9365153|
|     85|   1.0|    29| 0.27858746|
|     85|   1.0|     2| -1.1309202|
|     65|   1.0|    22|0.015822537|
|     65|   1.0|    16|  0.9821819|
|     65|   1.0|    19|  1.5534692|
|     78|   1.0|    12|  0.3426779|
|     78|   1.0|     8| 0.16658008|
|     78|   1.0|    11| 0.89204353|
|     34|   1.0|    19|-0.42913675|
|     34|   1.0|     4|  0.8540583|
|     81|   2.0|    29|  1.5044459|
|     28|   1.0|    17| -0.4737171|
|     28|   1.0|    23| 0.19614977|
|     28|   1.0|     2|  2.0880737|
|     76|   1.0|    19| -0.2536189|
|     76|   1.0|    11|  0.9034529|
|     76|   5.0|    14| 0.86341846|
|     76|   1.0|     2|  0.6112538|
+-------+------+------+-----------+
only showing top 20 rows



In [13]:
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

In [14]:
rmse = evaluator.evaluate(predictions)

In [15]:
rmse

1.9726879648621851

In [18]:
single_user = test.filter(test['userId']==12).select(['movieId', 'userId'])

In [19]:
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|      7|    12|
|     14|    12|
|     17|    12|
|     18|    12|
|     41|    12|
|     51|    12|
|     72|    12|
|     78|    12|
+-------+------+



In [20]:
recommendations = model.transform(single_user)

In [21]:
recommendations.orderBy('prediction', ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     51|    12|   4.834475|
|     18|    12|  4.6947904|
|      7|    12|  3.6062138|
|     17|    12|  1.8428576|
|     72|    12| 0.91749704|
|     14|    12|  0.5120641|
|     78|    12|  0.3426779|
|     41|    12|-0.15960929|
+-------+------+-----------+

